In [1]:
# imports
import os
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, regexp_replace, monotonically_increasing_id
from pyspark.sql.types import IntegerType, DecimalType, StringType

In [2]:
# iniciar spark
conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1a216ae4-cdba-4237-81fe-3a2cb6ff23eb;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 434ms :: artifacts dl 12ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	-------------------------------------------

In [3]:
# Variáveis S3
bucket_bruto = 'mario-sprint3-heleno-brutos'
bucket_tratado = 'mario-sprint3-heleno-tratados'

In [4]:
def upload_df_s3(df, bucket_name, file_name, type_file='csv'):
    if type_file == 'csv':
        df.write \
        .option("header", True) \
        .option("delimiter",";") \
        .mode('overwrite') \
        .csv(f's3a://{bucket_name}/{file_name}')
    elif type_file == 'json':
        df.write \
       .mode('overwrite') \
       .json(f's3a://{bucket_name}/{file_name}')
    else:
        print('Extensão de arquivo inválida')

In [5]:
def file_to_dataframe(bucket_name, file_name, delimiter=';', file_format='csv'):
    if file_format == 'csv':
        return spark.read.option('delimiter', delimiter).option('header', 'true').csv(f's3a://{bucket_name}/{file_name}')
    elif file_format == 'json':
        return spark.read.option("multiline","true").json(f's3a://{bucket_name}/{file_name}')
    else:
        print('Formato inválido')

In [6]:
# load s3 file
file_name = 'tweetsBrutos.json'

df = file_to_dataframe(bucket_bruto, file_name, file_format='json')

22/06/07 23:22:50 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
22/06/07 23:22:57 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [7]:
df.select('id', 'urls', 'text').distinct().show()

+-------------------+--------------------+--------------------+
|                 id|                urls|                text|
+-------------------+--------------------+--------------------+
|1531283007759429632|[{https://twitter...|Depois de fazer o...|
|1531424535723102214|[{https://twitter...|COMO ME CONQUISTA...|
|1532510451376041984|                  []|@Luucaax5 Sim ami...|
|1534290233671528454|[{https://twitter...|Todo cristão tem ...|
|1532871059090857986|[{https://twitter...|@halsierino Jdjdk...|
|1532148011257700352|[{https://twitter...|@miarspx altura: ...|
|1531424945200406529|                  []|RT @lwtnorris: nã...|
|1532509536912920576|                  []|RT @CarlosPSTU: @...|
|1534292160115351552|[{https://twitter...|cadê a carne http...|
|1532873823422402560|                  []|@teteuzada Não po...|
|1531424149603860484|[{https://twitter...|bolo: cenoura \ns...|
|1532509953893728272|                  []|Fui numa manicure...|
|1534028575317405696|[{https://twitter..

In [8]:
df.distinct().count()

698